In [1]:
## Knowing the annotations
img = io.imread(DATASET_PATH +"\\" +"000_0001.png")
show_images([img], ["Sample Image"])
print (img.shape)
#add bounding box coordinates
bounding_box = (19,7,120,117)
cv.circle(img, (bounding_box[0], bounding_box[1]), radius=1,color=(0, 255, 0), thickness=-1)
cv.circle(img, (bounding_box[2], bounding_box[3]), radius=1,color=(0, 255, 0), thickness=-1)
show_images([img], ["Sample Image"])
cv.rectangle(img, (bounding_box[0], bounding_box[1]), (bounding_box[2], bounding_box[3]), (0, 255, 0), 1)
show_images([img], ["Sample Image"])

# print the annotation file content
with open(ANNOTATION_FILE, 'r') as f:
    print(f.read())

NameError: name 'io' is not defined

In [2]:
# circle detectction:

 
import cv2 
import numpy as np 
  
# # Read image. 
# img = cv2.imread('TSRD_Train/022_0001_j.png', cv2.IMREAD_COLOR) 
  
# # Convert to grayscale. 
# gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) 

img = io.imread("TSRD_Train/022_0002_j.png")
gray = rgb2gray(img)
  
# Blur using 3 * 3 kernel. 
gray_blurred = gaussian(gray, 2)

show_images([img, gray, gray_blurred], ["Original", "Gray", "Blurred"])
  
# Apply Hough transform on the blurred image. 
detected_circles = cv2.HoughCircles(canny_edge_detection(gray),  
                   cv2.HOUGH_GRADIENT, 1, 20, param1 = 50, 
               param2 = 30, minRadius = 60, maxRadius = 90) 

show_images([canny_edge_detection(gray_blurred)], ["Edges"])
  
# Draw circles that are detected. 
if detected_circles is not None: 
    print ("detected_circles", detected_circles)
  
    # Convert the circle parameters a, b and r to integers. 
    detected_circles = np.uint16(np.around(detected_circles)) 
  
    for pt in detected_circles[0, :]: 
        a, b, r = pt[0], pt[1], pt[2] 
  
        # Draw the circumference of the circle. 
        cv2.circle(img, (a, b), r, (0, 255, 0), 2) 
  
        # Draw a small circle (of radius 1) to show the center. 
        cv2.circle(img, (a, b), 1, (0, 0, 255), 3) 
        cv2.imshow("Detected Circle", img) 

show_images([img], ["Detected Circle"])

In [1]:
# Circle and line

# Shape-Based Features (edge detection, Hough transform)

#canny edge detection
def canny_edge_detection(image):
    edges = cv.Canny((image * 255).astype(np.uint8), 40, 80)
    return edges

# Lines Hough transform
def lines_hough_transform(image):
    gray = rgb2gray(image)
    edges = canny_edge_detection(gray)
    lines = cv.HoughLinesP(edges, rho=1, theta=np.pi/180, threshold=45, minLineLength=20, maxLineGap=10)
    return lines

# Circles Hough transform
def circles_hough_transform(image):
    gray = rgb2gray(image)
    edges = canny_edge_detection(gray)
    circles = cv.HoughCircles(edges, cv.HOUGH_GRADIENT, dp=1.5, minDist=300, param1=50, param2=30, minRadius=110, maxRadius=200)
    return circles


# Apply Canny edge detection
edges = canny_edge_detection(img)

# Apply Hough transform for lines
lines = lines_hough_transform(img)
if lines is not None:
    for line in lines:
        x1, y1, x2, y2 = line[0]
        cv.line(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
        
# Apply Hough transform for circles
detected_circles = circles_hough_transform(img)
if detected_circles is not None: 
    print ("detected_circles", detected_circles)
    # Convert the circle parameters a, b and r to integers. 
    detected_circles = np.uint16(np.around(detected_circles)) 
    for pt in detected_circles[0, :]: 
        a, b, r = pt[0], pt[1], pt[2] 
        # Draw the circumference of the circle. 
        cv.circle(img, (a, b), r, (0, 255, 0), 2) 
        # Draw a small circle (of radius 1) to show the center. 
        cv.circle(img, (a, b), 1, (0, 0, 255), 3) 

In [2]:
# sift

img1 = images[2]
img2 = io.imread("TSRD_Train/022_0001_j.png")

show_images([img1, img2], ["Image 1", "Image 2"])

keypoints1, descriptors1 = sift_features(img1)
img1 = cv.drawKeypoints(img1, keypoints1, None)

keypoints2, descriptors2 = sift_features(img2)
img2 = cv.drawKeypoints(img2, keypoints2, None)

show_images([img1, img2], ["Image 1", "Image 2"])

## TODO: Construct BFMatcher, Hint use cv2.BFMatcher
## Use knnMatch with k = 2 to get the matches between the descriptors 
bf = cv.BFMatcher()
matches = bf.knnMatch(descriptors1,descriptors2, k=2)
# print (matches)

## TODO: Apply ratio test to remove ambigiuous matched (points that has multiple matches)
## Hint: use the distance in match 
good = []
for m,n in matches:
    if m.distance < 0.7*n.distance:
        good.append([m])

## For k=2 knnMatch return the 2 closest matches anyways.
## if the distance between the keypoints and its 2 neighbours is close, then the keypoint is ambigious and 
## you should neglect it

## TODO: Draw the matches features use cv2.drawMatchesKnn with flags =4 
img3 = cv.drawMatchesKnn(img1,keypoints1,img2,keypoints2,good,None,flags=4)
show_images([img3])

In [ ]:

# split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(images_histograms, labels, test_size=0.2, random_state=50)

# Evaluate Classifier
y_pred = svm.predict(X_test)
# show the prediction results
print("Prediction Results:")
print(y_test)
print(y_pred)

accuracy = accuracy_score(y_test, y_pred)
print(f"Test Accuracy: {accuracy * 100:.2f}%")